In [1]:
from datetime import datetime

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, split, when

In [2]:
# Create a SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Midares test job") \
    .getOrCreate()

24/05/06 14:55:43 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
24/05/06 14:55:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/06 14:55:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pjc_path= "/home/pengfei/git/PySparkCommonFunc/data/midares/PJC.csv"
pjc_df_1 = spark.read.csv(pjc_path,header=True, inferSchema=True)

In [4]:
pjc_df_1.printSchema()

In [5]:
print(pjc_df_1.show())